# Progetto Data Science Lab 2022/23

## Librerie

In [1]:
import pandas as pd
import json
import datetime
import tweepy
from datetime import datetime
from datetime import date
import matplotlib.pyplot as pl
import seaborn as sns
import numpy as np

## Import datasets e preprocessing

### Dataset generale e location ristoranti

In [2]:
df_ristorazione =  pd.read_csv('Serie-storiche-ristoranti_def.csv', header = 0)
df_ristorazione

,data,scontrini,lordototale,id_ristorante
0,20180101,21884.0,500409.9,R000
1,20180101,19961.0,252223.2,R003
2,20180101,19541.0,408110.4,R005
3,20180101,23173.0,516431.7,R004
4,20180101,28101.0,782775.9,R001
...,...,...,...,...
8812,20220430,0.0,0.0,R000
8813,20220430,0.0,0.0,R003
8814,20220430,0.0,0.0,R002
8815,20220430,0.0,0.0,R005


Importiamo il dataset di interesse, che mostra i dati dei guadagni di sei ristoranti in un periodo compreso tra il 01/01/2018 e il 30/04/2022.

Inoltre, abbiamo i dati relativi alla città in cui si trovano i ristoranti del dataset di interesse:

In [3]:
df_info_ristoranti =  pd.read_csv('RistorantiCitta.csv', header = 0, delimiter=';')
df_info_ristoranti

,Codice,Location
0,R000,Montebello
1,R003,Piacenza
2,R005,Voghera
3,R004,Stradella
4,R001,Piacenza
5,R002,Piacenza


In [6]:
df_ristorazione.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8817 entries, 0 to 8816
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   data           8817 non-null   int64  
 1   scontrini      8812 non-null   float64
 2   lordototale    8811 non-null   float64
 3   id_ristorante  8817 non-null   object 
dtypes: float64(2), int64(1), object(1)
memory usage: 275.7+ KB


In [8]:
df_info_ristoranti.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Codice    6 non-null      object
 1   Location  6 non-null      object
dtypes: object(2)
memory usage: 224.0+ bytes


### Informazioni relative alla location dei ristoranti 

In [9]:
# definizione regione e provincia

conditions = [
    (df_info_ristoranti['Location'] == 'Montebello'),
    (df_info_ristoranti['Location'] == 'Piacenza'),
    (df_info_ristoranti['Location'] == 'Voghera'),
    (df_info_ristoranti['Location'] == 'Stradella')]

province = ['Pavia', 'Piacenza', 'Pavia', 'Pavia']
regioni = ['Lombardia', 'Emilia-Romagna', 'Lombardia', 'Lombardia']

df_info_ristoranti['Provincia'] = np.select(conditions, province)
df_info_ristoranti['Regione'] = np.select(conditions, regioni)
df_info_ristoranti

,Codice,Location,Provincia,Regione
0,R000,Montebello,Pavia,Lombardia
1,R003,Piacenza,Piacenza,Emilia-Romagna
2,R005,Voghera,Pavia,Lombardia
3,R004,Stradella,Pavia,Lombardia
4,R001,Piacenza,Piacenza,Emilia-Romagna
5,R002,Piacenza,Piacenza,Emilia-Romagna


In [10]:
df_ristorazione = pd.merge(df_info_ristoranti, df_ristorazione, left_on='Codice', right_on='id_ristorante')

# Unione nel dataframe originale
df_ristorazione = df_ristorazione.drop('id_ristorante', axis=1)
df_ristorazione

,Codice,Location,Provincia,Regione,data,scontrini,lordototale
0,R000,Montebello,Pavia,Lombardia,20180101,21884.0,500409.900
1,R000,Montebello,Pavia,Lombardia,20180102,0.0,0.000
2,R000,Montebello,Pavia,Lombardia,20180103,0.0,0.000
3,R000,Montebello,Pavia,Lombardia,20180104,0.0,0.000
4,R000,Montebello,Pavia,Lombardia,20180105,0.0,0.000
...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,20220426,624.0,19369.152
8813,R002,Piacenza,Piacenza,Emilia-Romagna,20220427,0.0,0.000
8814,R002,Piacenza,Piacenza,Emilia-Romagna,20220428,723.0,24217.650
8815,R002,Piacenza,Piacenza,Emilia-Romagna,20220429,0.0,0.000


### Elaborazione date

Creiamo le colonne relative alla data

In [11]:
# formato colonna data
df_ristorazione['data'] = pd.to_datetime(df_ristorazione['data'], format='%Y%m%d')
df_ristorazione = df_ristorazione.rename(columns={'data': 'Data'})

# creazione colonna che indica il giorno della settimana
df_ristorazione['Giorno'] = df_ristorazione['Data'].dt.day_name()

# creazione colonna che indica il mese dell'anno
df_ristorazione['Mese'] = df_ristorazione['Data'].dt.month_name()

# creazione colonna che indica il mese dell'anno
df_ristorazione['Anno'] = df_ristorazione['Data'].dt.strftime('%Y')
df_ristorazione

,Codice,Location,Provincia,Regione,Data,scontrini,lordototale,Giorno,Mese,Anno
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884.0,500409.900,Monday,January,2018
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0.0,0.000,Tuesday,January,2018
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0.0,0.000,Wednesday,January,2018
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0.0,0.000,Thursday,January,2018
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0.0,0.000,Friday,January,2018
...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624.0,19369.152,Tuesday,April,2022
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0.0,0.000,Wednesday,April,2022
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723.0,24217.650,Thursday,April,2022
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0.0,0.000,Friday,April,2022


Inoltre, facciamo una divisione in stagioni

In [12]:
# divisione in stagioni

conditions = [
    (df_ristorazione['Mese'] == 'January'),
    (df_ristorazione['Mese'] == 'February'),
    (df_ristorazione['Mese'] == 'March') & (df_ristorazione['Data'].dt.day < 21),
    (df_ristorazione['Mese'] == 'March') & (df_ristorazione['Data'].dt.day >= 21),
    (df_ristorazione['Mese'] == "April"),
    (df_ristorazione['Mese'] == 'May'),
    (df_ristorazione['Mese'] == 'June') & (df_ristorazione['Data'].dt.day < 21),
    (df_ristorazione['Mese'] == 'June') & (df_ristorazione['Data'].dt.day >= 21),
    (df_ristorazione['Mese'] == "July"),
    (df_ristorazione['Mese'] == 'August'),
    (df_ristorazione['Mese'] == 'September') & (df_ristorazione['Data'].dt.day < 22),
    (df_ristorazione['Mese'] == 'September') & (df_ristorazione['Data'].dt.day >= 22),
    (df_ristorazione['Mese'] == "October"),
    (df_ristorazione['Mese'] == 'November'),
    (df_ristorazione['Mese'] == 'December') & (df_ristorazione['Data'].dt.day < 21),
    (df_ristorazione['Mese'] == 'December') & (df_ristorazione['Data'].dt.day >= 21),
    ]

values = ['Winter', 'Winter', 'Winter', 'Spring', 'Spring', 'Spring', 'Spring', 'Summer', 'Summer', 'Summer', 'Summer', 'Autumn', 'Autumn', 'Autumn', 'Autumn', 'Winter']

df_ristorazione['Stagione'] = np.select(conditions, values)
df_ristorazione

,Codice,Location,Provincia,Regione,Data,scontrini,lordototale,Giorno,Mese,Anno,Stagione
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884.0,500409.900,Monday,January,2018,Winter
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0.0,0.000,Tuesday,January,2018,Winter
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0.0,0.000,Wednesday,January,2018,Winter
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0.0,0.000,Thursday,January,2018,Winter
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0.0,0.000,Friday,January,2018,Winter
...,...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624.0,19369.152,Tuesday,April,2022,Spring
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0.0,0.000,Wednesday,April,2022,Spring
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723.0,24217.650,Thursday,April,2022,Spring
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0.0,0.000,Friday,April,2022,Spring


Successivamente, annotiamo quali giorni sono festivi (specificando la tipologia di festa) e quali feriali

In [13]:
# selezione giorni di festività

conditions = [
    (df_ristorazione['Mese'] == 'January') & (df_ristorazione['Data'].dt.day == 1),
    (df_ristorazione['Mese'] == 'January') & (df_ristorazione['Data'].dt.day == 6),
    (df_ristorazione['Mese'] == 'April') & (df_ristorazione['Data'].dt.day == 25),
    (df_ristorazione['Mese'] == 'May') & (df_ristorazione['Data'].dt.day == 1),
    (df_ristorazione['Mese'] == 'June') & (df_ristorazione['Data'].dt.day == 2),
    (df_ristorazione['Mese'] == 'August') & (df_ristorazione['Data'].dt.day == 15),
    (df_ristorazione['Mese'] == 'November') & (df_ristorazione['Data'].dt.day == 1),
    (df_ristorazione['Mese'] == 'December') & (df_ristorazione['Data'].dt.day == 8),
    (df_ristorazione['Mese'] == 'December') & (df_ristorazione['Data'].dt.day == 25),
    (df_ristorazione['Mese'] == 'December') & (df_ristorazione['Data'].dt.day == 26),
    (df_ristorazione['Data'] == '2018-04-01'),
    (df_ristorazione['Data'] == '2019-04-21'),
    (df_ristorazione['Data'] == '2020-04-12'),
    (df_ristorazione['Data'] == '2021-04-04'),
    (df_ristorazione['Data'] == '2022-04-17')
    ]

values = ['Capodanno', 'Epifania', 'Liberazione', 'Lavoro', 'Repubblica', 'Ferragosto', 'Ognissanti', 'Immacolata', 'Natale', 'Santo Stefano', 'Pasqua', 'Pasqua', 'Pasqua', 'Pasqua', 'Pasqua']

df_ristorazione['Festività'] = np.select(conditions, values)
df_ristorazione.loc[df_ristorazione["Festività"] == "0", "Festività"] = 'No'
df_ristorazione

,Codice,Location,Provincia,Regione,Data,scontrini,lordototale,Giorno,Mese,Anno,Stagione,Festività
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884.0,500409.900,Monday,January,2018,Winter,Capodanno
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0.0,0.000,Tuesday,January,2018,Winter,No
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0.0,0.000,Wednesday,January,2018,Winter,No
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0.0,0.000,Thursday,January,2018,Winter,No
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0.0,0.000,Friday,January,2018,Winter,No
...,...,...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624.0,19369.152,Tuesday,April,2022,Spring,No
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0.0,0.000,Wednesday,April,2022,Spring,No
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723.0,24217.650,Thursday,April,2022,Spring,No
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0.0,0.000,Friday,April,2022,Spring,No


In [14]:
# divisione giorni festivi e ferriali

conditions = [
    (df_ristorazione['Giorno'] == 'Saturday'),
    (df_ristorazione['Giorno'] == 'Sunday'),
    (df_ristorazione['Giorno'] != 'Saturday') & (df_ristorazione['Festività'] != 'No'),
    (df_ristorazione['Giorno'] != 'Sunday') & (df_ristorazione['Festività'] != 'No'),
    ]

values = ['Festivo', 'Festivo', 'Festivo', 'Festivo']

df_ristorazione['Festivo_feriale'] = np.select(conditions, values)
df_ristorazione.loc[df_ristorazione["Festivo_feriale"] == "0", "Festivo_feriale"] = 'Feriale'
df_ristorazione

,Codice,Location,Provincia,Regione,Data,scontrini,lordototale,Giorno,Mese,Anno,Stagione,Festività,Festivo_feriale
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884.0,500409.900,Monday,January,2018,Winter,Capodanno,Festivo
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0.0,0.000,Tuesday,January,2018,Winter,No,Feriale
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0.0,0.000,Wednesday,January,2018,Winter,No,Feriale
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0.0,0.000,Thursday,January,2018,Winter,No,Feriale
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0.0,0.000,Friday,January,2018,Winter,No,Feriale
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624.0,19369.152,Tuesday,April,2022,Spring,No,Feriale
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0.0,0.000,Wednesday,April,2022,Spring,No,Feriale
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723.0,24217.650,Thursday,April,2022,Spring,No,Feriale
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0.0,0.000,Friday,April,2022,Spring,No,Feriale


Infine, facciamo una divisione tra le date prima e dopo l'inizio del Covid-19 in Italia

In [15]:
conditions = [
    (df_ristorazione['Data'] < '2020-02-21'),
    (df_ristorazione['Data'] >= '2020-02-21')
    ]

values = ['Pre-Covid19', 'Post-Covid19']

df_ristorazione['Covid'] = np.select(conditions, values)
df_ristorazione

,Codice,Location,Provincia,Regione,Data,scontrini,lordototale,Giorno,Mese,Anno,Stagione,Festività,Festivo_feriale,Covid
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884.0,500409.900,Monday,January,2018,Winter,Capodanno,Festivo,Pre-Covid19
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0.0,0.000,Tuesday,January,2018,Winter,No,Feriale,Pre-Covid19
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0.0,0.000,Wednesday,January,2018,Winter,No,Feriale,Pre-Covid19
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0.0,0.000,Thursday,January,2018,Winter,No,Feriale,Pre-Covid19
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0.0,0.000,Friday,January,2018,Winter,No,Feriale,Pre-Covid19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624.0,19369.152,Tuesday,April,2022,Spring,No,Feriale,Post-Covid19
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0.0,0.000,Wednesday,April,2022,Spring,No,Feriale,Post-Covid19
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723.0,24217.650,Thursday,April,2022,Spring,No,Feriale,Post-Covid19
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0.0,0.000,Friday,April,2022,Spring,No,Feriale,Post-Covid19


### Dataset casi covid

In [16]:
df_casicovid = pd.read_json('dpc-covid19-ita-province.json')
df_casicovid['data'] = pd.to_datetime(df_casicovid['data']).dt.date
df_casicovid['data'] = pd.to_datetime(df_casicovid['data'])

df_casicovid

,data,stato,codice_regione,denominazione_regione,codice_provincia,denominazione_provincia,sigla_provincia,lat,long,totale_casi,note,codice_nuts_1,codice_nuts_2,codice_nuts_3
0,2020-02-24,ITA,13,Abruzzo,66,L'Aquila,AQ,42.351222,13.398438,0,None,None,None,None
1,2020-02-24,ITA,13,Abruzzo,67,Teramo,TE,42.658918,13.704400,0,None,None,None,None
2,2020-02-24,ITA,13,Abruzzo,68,Pescara,PE,42.464584,14.213648,0,None,None,None,None
3,2020-02-24,ITA,13,Abruzzo,69,Chieti,CH,42.351032,14.167546,0,None,None,None,None
4,2020-02-24,ITA,13,Abruzzo,979,In fase di definizione/aggiornamento,None,NaN,NaN,0,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172210,2023-05-11,ITA,5,Veneto,27,Venezia,VE,45.434905,12.338452,460504,None,ITH,ITH3,ITH35
172211,2023-05-11,ITA,5,Veneto,28,Padova,PD,45.406930,11.876087,537300,None,ITH,ITH3,ITH36
172212,2023-05-11,ITA,5,Veneto,29,Rovigo,RO,45.071073,11.790070,121308,None,ITH,ITH3,ITH37
172213,2023-05-11,ITA,5,Veneto,899,Fuori Regione / Provincia Autonoma,None,NaN,NaN,39430,None,ITH,ITH3,None


Rimuoviamo tutte le righe che non ci interessano: quelle relative a province diverse da Pavia e Piacenza, e quelle comprese tra il 2018-01-01 e il 2022-04-30

In [17]:
df_casicovid = df_casicovid[df_casicovid['data'] < '2022-05-01']
df_casicovid = df_casicovid[df_casicovid['denominazione_provincia'].isin(['Pavia', 'Piacenza'])]
df_casicovid

,data,stato,codice_regione,denominazione_regione,codice_provincia,denominazione_provincia,sigla_provincia,lat,long,totale_casi,note,codice_nuts_1,codice_nuts_2,codice_nuts_3
20,2020-02-24,ITA,8,Emilia-Romagna,33,Piacenza,PC,45.051935,9.692633,0,None,None,None,None
52,2020-02-24,ITA,3,Lombardia,18,Pavia,PV,45.185093,9.160157,0,None,None,None,None
148,2020-02-25,ITA,8,Emilia-Romagna,33,Piacenza,PC,45.051935,9.692633,18,None,None,None,None
180,2020-02-25,ITA,3,Lombardia,18,Pavia,PV,45.185093,9.160157,27,None,None,None,None
276,2020-02-26,ITA,8,Emilia-Romagna,33,Piacenza,PC,45.051935,9.692633,28,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115804,2022-04-28,ITA,3,Lombardia,18,Pavia,PV,45.185093,9.160157,145343,None,ITC,ITC4,ITC48
115917,2022-04-29,ITA,8,Emilia-Romagna,33,Piacenza,PC,45.051935,9.692633,72488,None,ITH,ITH5,ITH51
115953,2022-04-29,ITA,3,Lombardia,18,Pavia,PV,45.185093,9.160157,145811,None,ITC,ITC4,ITC48
116066,2022-04-30,ITA,8,Emilia-Romagna,33,Piacenza,PC,45.051935,9.692633,72676,None,ITH,ITH5,ITH51


Rimuoviamo le colonne non di interesse per l'analisi: latitudine, longitudine, codice_regione, codice_provincia, sigla_provincia, note, codice_nuts_1, codice_nuts_2, codice_nuts_3

In [18]:
df_casicovid = df_casicovid.drop(['lat', 'long', 'codice_regione', 'codice_provincia', 'sigla_provincia', 'note', 'codice_nuts_1', 'codice_nuts_2', 'codice_nuts_3'], axis=1)
df_casicovid


,data,stato,denominazione_regione,denominazione_provincia,totale_casi
20,2020-02-24,ITA,Emilia-Romagna,Piacenza,0
52,2020-02-24,ITA,Lombardia,Pavia,0
148,2020-02-25,ITA,Emilia-Romagna,Piacenza,18
180,2020-02-25,ITA,Lombardia,Pavia,27
276,2020-02-26,ITA,Emilia-Romagna,Piacenza,28
...,...,...,...,...,...
115804,2022-04-28,ITA,Lombardia,Pavia,145343
115917,2022-04-29,ITA,Emilia-Romagna,Piacenza,72488
115953,2022-04-29,ITA,Lombardia,Pavia,145811
116066,2022-04-30,ITA,Emilia-Romagna,Piacenza,72676


In [19]:
# verifica missing values
df_casicovid.isnull().sum()

data                       0
stato                      0
denominazione_regione      0
denominazione_provincia    0
totale_casi                0
dtype: int64

In [20]:
# modifica nomi colonne
df_casicovid = df_casicovid.rename(columns={'data': 'Data', 'stato': 'Stato', 'denominazione_regione': 
                                            'Denominazione_regione', 'denominazione_provincia': 'Denominazione_provincia', 
                                            'totale_casi': 'Totale_casi'})
df_casicovid

,Data,Stato,Denominazione_regione,Denominazione_provincia,Totale_casi
20,2020-02-24,ITA,Emilia-Romagna,Piacenza,0
52,2020-02-24,ITA,Lombardia,Pavia,0
148,2020-02-25,ITA,Emilia-Romagna,Piacenza,18
180,2020-02-25,ITA,Lombardia,Pavia,27
276,2020-02-26,ITA,Emilia-Romagna,Piacenza,28
...,...,...,...,...,...
115804,2022-04-28,ITA,Lombardia,Pavia,145343
115917,2022-04-29,ITA,Emilia-Romagna,Piacenza,72488
115953,2022-04-29,ITA,Lombardia,Pavia,145811
116066,2022-04-30,ITA,Emilia-Romagna,Piacenza,72676


### Dataset vaccinazioni

In [21]:
df_vaccinazioni = pd.read_csv('italian_vaccination.csv')
df_vaccinazioni['administration_date'] = pd.to_datetime(df_vaccinazioni['administration_date'])
df_vaccinazioni

,administration_date,supplier,region,age_range,males,females,first_dose,second_dose,previous_infection,additional_booster_dose,second_booster,db3,NUTS1_code,NUTS2_code,ISTAT_regional_code,region_name
0,2020-12-27,Moderna,LAZ,40-49,0,1,1,0,0,0,0,0,ITI,ITI4,12,Lazio
1,2020-12-27,Moderna,SIC,90+,0,1,1,0,0,0,0,0,ITG,ITG1,19,Sicilia
2,2020-12-27,Pfizer/BioNTech,ABR,20-29,1,1,2,0,0,0,0,0,ITF,ITF1,13,Abruzzo
3,2020-12-27,Pfizer/BioNTech,ABR,30-39,1,4,5,0,0,0,0,0,ITF,ITF1,13,Abruzzo
4,2020-12-27,Pfizer/BioNTech,ABR,40-49,1,8,9,0,0,0,0,0,ITF,ITF1,13,Abruzzo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301425,2023-05-15,Pfizer/BioNTech,TOS,90+,0,1,0,0,0,0,0,1,ITI,ITI1,9,Toscana
301426,2023-05-15,Pfizer/BioNTech,UMB,12-19,0,1,0,1,0,0,0,0,ITI,ITI2,10,Umbria
301427,2023-05-15,Pfizer/BioNTech,VEN,50-59,0,1,0,0,0,0,1,0,ITH,ITH3,5,Veneto
301428,2023-05-15,Pfizer/BioNTech,VEN,60-69,1,1,0,0,0,1,0,1,ITH,ITH3,5,Veneto


Rimuoviamo le colonne non di interesse: supplier, age_range, db3, NUTS1_code, NUTS2_code, ISTAT_regional_code.

In [22]:
df_vaccinazioni = df_vaccinazioni.drop(['region', 'supplier', 'age_range', 'db3', 'NUTS1_code', 'NUTS2_code', 'ISTAT_regional_code'], axis=1)
df_vaccinazioni

,administration_date,males,females,first_dose,second_dose,previous_infection,additional_booster_dose,second_booster,region_name
0,2020-12-27,0,1,1,0,0,0,0,Lazio
1,2020-12-27,0,1,1,0,0,0,0,Sicilia
2,2020-12-27,1,1,2,0,0,0,0,Abruzzo
3,2020-12-27,1,4,5,0,0,0,0,Abruzzo
4,2020-12-27,1,8,9,0,0,0,0,Abruzzo
...,...,...,...,...,...,...,...,...,...
301425,2023-05-15,0,1,0,0,0,0,0,Toscana
301426,2023-05-15,0,1,0,1,0,0,0,Umbria
301427,2023-05-15,0,1,0,0,0,0,1,Veneto
301428,2023-05-15,1,1,0,0,0,1,0,Veneto


Rimuoviamo tutte le righe che non ci interessano: quelle relative a regioni diverse da Lombardia ed Emilia-Romagna, e quelle comprese tra il 2018-01-01 e il 2022-04-30.

In [23]:
df_vaccinazioni = df_vaccinazioni[df_vaccinazioni['administration_date'] < '2022-05-01']
df_vaccinazioni = df_vaccinazioni[df_vaccinazioni['region_name'].isin(['Emilia-Romagna', 'Lombardia'])]
df_vaccinazioni

,administration_date,males,females,first_dose,second_dose,previous_infection,additional_booster_dose,second_booster,region_name
25,2020-12-27,53,86,139,0,0,0,0,Emilia-Romagna
26,2020-12-27,58,83,141,0,0,0,0,Emilia-Romagna
27,2020-12-27,80,148,228,0,0,0,0,Emilia-Romagna
28,2020-12-27,71,217,288,0,0,0,0,Emilia-Romagna
29,2020-12-27,100,51,151,0,0,0,0,Emilia-Romagna
...,...,...,...,...,...,...,...,...,...
219188,2022-04-30,221,210,6,17,2,380,26,Lombardia
219189,2022-04-30,184,159,9,7,0,214,113,Lombardia
219190,2022-04-30,149,159,2,8,0,119,179,Lombardia
219191,2022-04-30,1132,1350,3,3,1,59,2416,Lombardia


Ora unifichiamo con un *group by* i dati delle vaccinazioni, ottenendo una riga per ogni data e per entrambe le regioni. 

In [24]:
df_vaccinazioni = df_vaccinazioni.groupby(['administration_date', 'region_name']).sum().reset_index()
df_vaccinazioni

,administration_date,region_name,males,females,first_dose,second_dose,previous_infection,additional_booster_dose,second_booster
0,2020-12-27,Emilia-Romagna,374,599,973,0,0,0,0
1,2020-12-27,Lombardia,414,305,718,0,1,0,0
2,2020-12-28,Emilia-Romagna,1,0,1,0,0,0,0
3,2020-12-28,Lombardia,47,61,108,0,0,0,0
4,2020-12-29,Lombardia,45,62,107,0,0,0,0
...,...,...,...,...,...,...,...,...,...
973,2022-04-28,Lombardia,3616,4328,101,222,36,2483,5102
974,2022-04-29,Emilia-Romagna,3147,3291,73,152,40,976,5197
975,2022-04-29,Lombardia,3825,4642,142,226,81,2711,5307
976,2022-04-30,Emilia-Romagna,2412,2676,70,171,60,1149,3638


In [25]:
# modifica nomi colonne
df_vaccinazioni = df_vaccinazioni.rename(columns={'administration_date': 'Data_amministrativa', 'region_name': 
                                                  'Regione', 'males': 'Maschi', 'females': 'Femmine', 
                                                  'first_dose': 'Prima_dose', 'second_dose': 'Seconda_dose', 
                                                  'previous_infection': 'Contagio_precedente', 'additional_booster_dose': 
                                                  'Dose_Booster', 'second_booster': 'Seconda_dose_Booster'})
df_vaccinazioni

,Data_amministrativa,Regione,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster
0,2020-12-27,Emilia-Romagna,374,599,973,0,0,0,0
1,2020-12-27,Lombardia,414,305,718,0,1,0,0
2,2020-12-28,Emilia-Romagna,1,0,1,0,0,0,0
3,2020-12-28,Lombardia,47,61,108,0,0,0,0
4,2020-12-29,Lombardia,45,62,107,0,0,0,0
...,...,...,...,...,...,...,...,...,...
973,2022-04-28,Lombardia,3616,4328,101,222,36,2483,5102
974,2022-04-29,Emilia-Romagna,3147,3291,73,152,40,976,5197
975,2022-04-29,Lombardia,3825,4642,142,226,81,2711,5307
976,2022-04-30,Emilia-Romagna,2412,2676,70,171,60,1149,3638


### Dataset inflazione

In [26]:
df_inflazione = pd.read_excel('inflaz.xlsx')
df_inflazione

,TIME,All-items HICP,Food,"Electricity, gas and other fuels"
0,TIME,NaN,NaN,NaN
1,2018-01,1.2,1.2,5.9
2,2018-02,0.5,-1.0,4.9
3,2018-03,0.9,0.5,4.6
4,2018-04,0.6,1.4,-1.0
...,...,...,...,...
71,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN
74,Special value,NaN,NaN,NaN


In [27]:
df_inflazione.tail(12)

,TIME,All-items HICP,Food,"Electricity, gas and other fuels"
64,2023-04,8.7,12.2,21.5
65,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN
67,NaN,NaN,NaN,NaN
68,NaN,NaN,NaN,NaN
69,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN


La prima e le ultime 11 righe presentano problemi, le cancelliamo

In [28]:
df_inflazione = df_inflazione.drop(df_inflazione.tail(11).index)
df_inflazione = df_inflazione.drop(df_inflazione.index[0])
df_inflazione

,TIME,All-items HICP,Food,"Electricity, gas and other fuels"
1,2018-01,1.2,1.2,5.9
2,2018-02,0.5,-1.0,4.9
3,2018-03,0.9,0.5,4.6
4,2018-04,0.6,1.4,-1.0
5,2018-05,1,1.9,-0.9
...,...,...,...,...
60,2022-12,12.3,13.3,124.6
61,2023-01,10.7,12.6,67.3
62,2023-02,9.8,13.5,40.7
63,2023-03,8.1,13.3,17.3


Rimuoviamo tutte le righe che non ci interessano: quelle non comprese tra il 2018-01-01 e il 2022-04-30

In [29]:
df_inflazione = df_inflazione.rename(columns={'TIME': 'data', 'All-items HICP': 'Inflation'})
df_inflazione = df_inflazione[df_inflazione['data'] < '2022-05']
df_inflazione

,data,Inflation,Food,"Electricity, gas and other fuels"
1,2018-01,1.2,1.2,5.9
2,2018-02,0.5,-1.0,4.9
3,2018-03,0.9,0.5,4.6
4,2018-04,0.6,1.4,-1.0
5,2018-05,1,1.9,-0.9
6,2018-06,1.4,2.5,-0.8
7,2018-07,1.9,2.3,5.3
8,2018-08,1.6,2.1,5.3
9,2018-09,1.5,1.6,5.4
10,2018-10,1.7,0.6,10.0


Il seguente dataframe presenta i dati relativi all'inflazione in italia, in particolare riguardo il periodo di interesse compreso tra il 01/01/2018 e il 30/04/2022.

Sono presenti i valori dell'inflazione, dell'inflazione relativa al prezzo del cibo, e dell'inflazione relativa ad elettricità, gas e altri combustibili (come possibile conseguenza della guerra in Ucraina)

In [30]:
# Formato data per la colonna 'data'
df_inflazione['data'] = pd.to_datetime(df_inflazione['data'])

# creazione colonna che indica il mese dell'anno
df_inflazione['Mese'] = df_inflazione['data'].dt.month_name()

# creazione colonna che indica il mese dell'anno
df_inflazione['Anno'] = df_inflazione['data'].dt.strftime('%Y')
df_inflazione

,data,Inflation,Food,"Electricity, gas and other fuels",Mese,Anno
1,2018-01-01,1.2,1.2,5.9,January,2018
2,2018-02-01,0.5,-1.0,4.9,February,2018
3,2018-03-01,0.9,0.5,4.6,March,2018
4,2018-04-01,0.6,1.4,-1.0,April,2018
5,2018-05-01,1,1.9,-0.9,May,2018
6,2018-06-01,1.4,2.5,-0.8,June,2018
7,2018-07-01,1.9,2.3,5.3,July,2018
8,2018-08-01,1.6,2.1,5.3,August,2018
9,2018-09-01,1.5,1.6,5.4,September,2018
10,2018-10-01,1.7,0.6,10.0,October,2018


In [31]:
# modifica nomi colonne
df_inflazione = df_inflazione.rename(columns={'data': 'Mese e Anno', 'Inflation': 
                                                  'Inflazione', 'Food': 'Inflazione_cibo', 'Electricity, gas and other fuels': 
                                                  'Inflazione_combustibili'})
df_inflazione

,Mese e Anno,Inflazione,Inflazione_cibo,Inflazione_combustibili,Mese,Anno
1,2018-01-01,1.2,1.2,5.9,January,2018
2,2018-02-01,0.5,-1.0,4.9,February,2018
3,2018-03-01,0.9,0.5,4.6,March,2018
4,2018-04-01,0.6,1.4,-1.0,April,2018
5,2018-05-01,1,1.9,-0.9,May,2018
6,2018-06-01,1.4,2.5,-0.8,June,2018
7,2018-07-01,1.9,2.3,5.3,July,2018
8,2018-08-01,1.6,2.1,5.3,August,2018
9,2018-09-01,1.5,1.6,5.4,September,2018
10,2018-10-01,1.7,0.6,10.0,October,2018


### Dataset zone Covid

In [32]:
df_colori = pd.read_csv('colori.csv')
df_colori['data'] = pd.to_datetime(df_colori['data'])
df_colori

,data,denominazione_regione,colore
0,2020-11-06,Abruzzo,giallo
1,2020-11-07,Abruzzo,giallo
2,2020-11-08,Abruzzo,giallo
3,2020-11-09,Abruzzo,giallo
4,2020-11-10,Abruzzo,arancione
...,...,...,...
11671,2022-05-11,Veneto,bianco
11672,2022-05-12,Veneto,bianco
11673,2022-05-13,Veneto,bianco
11674,2022-05-14,Veneto,bianco


Rimuoviamo tutte le righe che non ci interessano: quelle relative a province diverse da Pavia e Piacenza, e quelle non comprese tra il 2018-01-01 e il 2022-04-30

In [33]:
df_colori = df_colori[df_colori['data'] < '2022-05-01']
df_colori = df_colori[df_colori['denominazione_regione'].isin(['Lombardia', 'Emilia-Romagna'])]
df_colori

,data,denominazione_regione,colore
2224,2020-11-06,Emilia-Romagna,giallo
2225,2020-11-07,Emilia-Romagna,giallo
2226,2020-11-08,Emilia-Romagna,giallo
2227,2020-11-09,Emilia-Romagna,giallo
2228,2020-11-10,Emilia-Romagna,giallo
...,...,...,...
4984,2022-04-26,Lombardia,bianco
4985,2022-04-27,Lombardia,bianco
4986,2022-04-28,Lombardia,bianco
4987,2022-04-29,Lombardia,bianco


Notiamo comunque che si tratta di informazioni che sono comprese tra il 06/11/2020 e il 30/04/2022, in quanto è dal 06/11/2020 che vengono ufficialmente istituite le zone di colore differente nei territori italiani in base al numero di casi.

Per il periodo compreso tra il 21/02/2020 e il 06/11/2020 inseriamo i seguenti dati all'interno del dataset:
- zona bianca nei periodi tra 21/02/2020 e 29/02/2020, e tra 11/06/2020 e 12/10/2020
- zona gialla nei periodi tra 04/05/2020 e 17/05/2020, e tra 13/10/2020 e 17/10/2020
- zona arancione nei periodi tra 18/05/2020 e 10/06/2020, e tra 18/10/2020 e 02/11/2020
- zona rossa nei periodi tra 01/03/2020 e 03/05/2020, e tra 03/11/2020 e 06/11/2020

In [34]:
pd.to_datetime('11/06/2020') - pd.to_datetime('02/21/2020')

Timedelta('259 days 00:00:00')

Sono 259 giorni, con informazioni relative a due regioni, dunque *259x2*

In [35]:
num_empty_rows = 259
empty_rows_l = pd.DataFrame({'data': [None] * num_empty_rows,
                   'denominazione_regione': 'Lombardia',
                   'colore': [None] * num_empty_rows})
start_date = '2020-02-21'
date_range = pd.date_range(start=start_date, periods=num_empty_rows)
empty_rows_l.loc[:, 'data'] = date_range
empty_rows_l['data'] = pd.to_datetime(empty_rows_l['data'])

C:\Users\andre\AppData\Local\Temp\ipykernel_89568\601435974.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  empty_rows_l.loc[:, 'data'] = date_range


In [36]:
num_empty_rows = 259
empty_rows_er = pd.DataFrame({'data': [None] * num_empty_rows,
                   'denominazione_regione': 'Emilia-Romagna',
                   'colore': [None] * num_empty_rows})
start_date = '2020-02-21'
date_range = pd.date_range(start=start_date, periods=num_empty_rows)
empty_rows_er.loc[:, 'data'] = date_range

C:\Users\andre\AppData\Local\Temp\ipykernel_89568\3301124254.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  empty_rows_er.loc[:, 'data'] = date_range


In [37]:
# modifica dati colori Lombardia per date tra 21/02/2020 e 06/11/2020

conditions = [
    (empty_rows_l['data'].between('2020-02-21','2020-02-29')),
    (empty_rows_l['data'].between('2020-03-01','2020-05-03')),
    (empty_rows_l['data'].between('2020-05-04','2020-05-17')),
    (empty_rows_l['data'].between('2020-05-18','2020-06-10')),
    (empty_rows_l['data'].between('2020-06-11','2020-10-12')),
    (empty_rows_l['data'].between('2020-10-13','2020-10-17')),
    (empty_rows_l['data'].between('2020-10-18','2020-11-02')),
    (empty_rows_l['data'].between('2020-11-03','2020-11-06'))
    ]

values = ['bianco', 'rosso', 'arancione', 'giallo', 'bianco', 'giallo', 'arancione', 'rosso']

empty_rows_l['colore'] = np.select(conditions, values)
empty_rows_l

,data,denominazione_regione,colore
0,2020-02-21,Lombardia,bianco
1,2020-02-22,Lombardia,bianco
2,2020-02-23,Lombardia,bianco
3,2020-02-24,Lombardia,bianco
4,2020-02-25,Lombardia,bianco
...,...,...,...
254,2020-11-01,Lombardia,arancione
255,2020-11-02,Lombardia,arancione
256,2020-11-03,Lombardia,rosso
257,2020-11-04,Lombardia,rosso


In [38]:
# modifica dati colori Emilia-Romagna per date tra 21/02/2020 e 06/11/2020

conditions = [
    (empty_rows_er['data'].between('2020-02-21','2020-02-29')),
    (empty_rows_er['data'].between('2020-03-01','2020-05-03')),
    (empty_rows_er['data'].between('2020-05-04','2020-05-17')),
    (empty_rows_er['data'].between('2020-05-18','2020-06-10')),
    (empty_rows_er['data'].between('2020-06-11','2020-10-12')),
    (empty_rows_er['data'].between('2020-10-13','2020-10-17')),
    (empty_rows_er['data'].between('2020-10-18','2020-11-02')),
    (empty_rows_er['data'].between('2020-11-03','2020-11-06'))
    ]

values = ['bianco', 'rosso', 'arancione', 'giallo', 'bianco', 'giallo', 'arancione', 'rosso']

empty_rows_er['colore'] = np.select(conditions, values)
empty_rows_er

,data,denominazione_regione,colore
0,2020-02-21,Emilia-Romagna,bianco
1,2020-02-22,Emilia-Romagna,bianco
2,2020-02-23,Emilia-Romagna,bianco
3,2020-02-24,Emilia-Romagna,bianco
4,2020-02-25,Emilia-Romagna,bianco
...,...,...,...
254,2020-11-01,Emilia-Romagna,arancione
255,2020-11-02,Emilia-Romagna,arancione
256,2020-11-03,Emilia-Romagna,rosso
257,2020-11-04,Emilia-Romagna,rosso


In [39]:
df_colori = pd.concat([empty_rows_l,empty_rows_er, df_colori], ignore_index=True)
df_colori

,data,denominazione_regione,colore
0,2020-02-21,Lombardia,bianco
1,2020-02-22,Lombardia,bianco
2,2020-02-23,Lombardia,bianco
3,2020-02-24,Lombardia,bianco
4,2020-02-25,Lombardia,bianco
...,...,...,...
1595,2022-04-26,Lombardia,bianco
1596,2022-04-27,Lombardia,bianco
1597,2022-04-28,Lombardia,bianco
1598,2022-04-29,Lombardia,bianco


In [40]:
df_colori.isna().sum()

data                     0
denominazione_regione    0
colore                   0
dtype: int64

In [41]:
# modifica nomi colonne
df_colori = df_colori.rename(columns={'data': 'Data', 'denominazione_regione': 'Regione', 'colore': 'Colore'})
df_colori

,Data,Regione,Colore
0,2020-02-21,Lombardia,bianco
1,2020-02-22,Lombardia,bianco
2,2020-02-23,Lombardia,bianco
3,2020-02-24,Lombardia,bianco
4,2020-02-25,Lombardia,bianco
...,...,...,...
1595,2022-04-26,Lombardia,bianco
1596,2022-04-27,Lombardia,bianco
1597,2022-04-28,Lombardia,bianco
1598,2022-04-29,Lombardia,bianco


### Integrazione dataframe completo

Ora uniamo le informazioni relative alle zone con i dati del nostro dataset originale.
Inoltre, specifichiamo per quali informazioni relative ai ristoranti eravamo in periodo precedente al Covid, dunque non ci sono indicazioni relative al colore della regione. 

In [42]:
df_ristorazione = df_ristorazione.merge(df_colori, how='left', left_on=['Data', 'Regione'], right_on=['Data', 'Regione'])

for index, row in df_ristorazione.iterrows():
    if row['Covid'] == 'Pre-Covid19':
        df_ristorazione.at[index, 'Colore'] = 'Pre-Covid19'
df_ristorazione

,Codice,Location,Provincia,Regione,Data,scontrini,lordototale,Giorno,Mese,Anno,Stagione,Festività,Festivo_feriale,Covid,Colore
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884.0,500409.900,Monday,January,2018,Winter,Capodanno,Festivo,Pre-Covid19,Pre-Covid19
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0.0,0.000,Tuesday,January,2018,Winter,No,Feriale,Pre-Covid19,Pre-Covid19
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0.0,0.000,Wednesday,January,2018,Winter,No,Feriale,Pre-Covid19,Pre-Covid19
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0.0,0.000,Thursday,January,2018,Winter,No,Feriale,Pre-Covid19,Pre-Covid19
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0.0,0.000,Friday,January,2018,Winter,No,Feriale,Pre-Covid19,Pre-Covid19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624.0,19369.152,Tuesday,April,2022,Spring,No,Feriale,Post-Covid19,bianco
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0.0,0.000,Wednesday,April,2022,Spring,No,Feriale,Post-Covid19,bianco
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723.0,24217.650,Thursday,April,2022,Spring,No,Feriale,Post-Covid19,bianco
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0.0,0.000,Friday,April,2022,Spring,No,Feriale,Post-Covid19,bianco


Successivamente uniamo le informazioni relative all'inflazione con i dati del nostro dataset originale.

In [43]:
df_ristorazione = df_ristorazione.merge(df_inflazione, how='left', left_on=['Mese', 'Anno'], right_on=['Mese', 'Anno'])
df_ristorazione = df_ristorazione.drop('Mese e Anno', axis=1)

df_ristorazione

,Codice,Location,Provincia,Regione,Data,scontrini,lordototale,Giorno,Mese,Anno,Stagione,Festività,Festivo_feriale,Covid,Colore,Inflazione,Inflazione_cibo,Inflazione_combustibili
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884.0,500409.900,Monday,January,2018,Winter,Capodanno,Festivo,Pre-Covid19,Pre-Covid19,1.2,1.2,5.9
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0.0,0.000,Tuesday,January,2018,Winter,No,Feriale,Pre-Covid19,Pre-Covid19,1.2,1.2,5.9
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0.0,0.000,Wednesday,January,2018,Winter,No,Feriale,Pre-Covid19,Pre-Covid19,1.2,1.2,5.9
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0.0,0.000,Thursday,January,2018,Winter,No,Feriale,Pre-Covid19,Pre-Covid19,1.2,1.2,5.9
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0.0,0.000,Friday,January,2018,Winter,No,Feriale,Pre-Covid19,Pre-Covid19,1.2,1.2,5.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624.0,19369.152,Tuesday,April,2022,Spring,No,Feriale,Post-Covid19,bianco,6.3,6.7,60.5
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0.0,0.000,Wednesday,April,2022,Spring,No,Feriale,Post-Covid19,bianco,6.3,6.7,60.5
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723.0,24217.650,Thursday,April,2022,Spring,No,Feriale,Post-Covid19,bianco,6.3,6.7,60.5
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0.0,0.000,Friday,April,2022,Spring,No,Feriale,Post-Covid19,bianco,6.3,6.7,60.5


Inoltre, uniamo le informazioni relative alle vaccinazioni con i dati del nostro dataset originale.
Poniamo tutti i dati inseriti come degli interi, e inseriamo valori pari a 0 per tutti i dati definiti come *NaN* essendo precedenti all'inizio della somministrazione dei vaccini

In [44]:
df_ristorazione = df_ristorazione.merge(df_vaccinazioni, how='left', left_on=['Data', 'Regione'], right_on=['Data_amministrativa', 'Regione'])
df_ristorazione = df_ristorazione.drop('Data_amministrativa', axis=1)

df_ristorazione[['Maschi', 'Femmine', 'Prima_dose', 'Seconda_dose', 'Contagio_precedente', 'Dose_Booster', 
    'Seconda_dose_Booster']] = df_ristorazione[['Maschi', 'Femmine', 'Prima_dose', 'Seconda_dose', 'Contagio_precedente', 
                                  'Dose_Booster', 'Seconda_dose_Booster']].fillna(0)

df_ristorazione[['Maschi', 'Femmine', 'Prima_dose', 'Seconda_dose', 'Contagio_precedente', 'Dose_Booster', 
    'Seconda_dose_Booster']] = df_ristorazione[['Maschi', 'Femmine', 'Prima_dose', 'Seconda_dose', 'Contagio_precedente', 
                                  'Dose_Booster', 'Seconda_dose_Booster']].astype(int)

df_ristorazione

,Codice,Location,Provincia,Regione,Data,scontrini,lordototale,Giorno,Mese,Anno,...,Inflazione,Inflazione_cibo,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884.0,500409.900,Monday,January,2018,...,1.2,1.2,5.9,0,0,0,0,0,0,0
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0.0,0.000,Tuesday,January,2018,...,1.2,1.2,5.9,0,0,0,0,0,0,0
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0.0,0.000,Wednesday,January,2018,...,1.2,1.2,5.9,0,0,0,0,0,0,0
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0.0,0.000,Thursday,January,2018,...,1.2,1.2,5.9,0,0,0,0,0,0,0
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0.0,0.000,Friday,January,2018,...,1.2,1.2,5.9,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624.0,19369.152,Tuesday,April,2022,...,6.3,6.7,60.5,3614,3915,86,262,62,1469,5650
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0.0,0.000,Wednesday,April,2022,...,6.3,6.7,60.5,3592,3804,71,193,27,1160,5945
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723.0,24217.650,Thursday,April,2022,...,6.3,6.7,60.5,3838,4031,77,234,29,1528,6001
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0.0,0.000,Friday,April,2022,...,6.3,6.7,60.5,3147,3291,73,152,40,976,5197


Infine, uniamo le informazioni relative ai casi Covid nelle singole date per ogni provincia con i dati del nostro dataset originale.

In [45]:
df_ristorazione = df_ristorazione.merge(df_casicovid, how='left', left_on=['Data', 'Provincia'], right_on=['Data', 'Denominazione_provincia'])
df_ristorazione = df_ristorazione.drop(['Stato', 'Denominazione_regione', 'Denominazione_provincia'], axis=1)

df_ristorazione['Totale_casi'] = df_ristorazione['Totale_casi'].fillna(0)
df_ristorazione['Totale_casi'] = df_ristorazione['Totale_casi'].astype(int)

df_ristorazione

,Codice,Location,Provincia,Regione,Data,scontrini,lordototale,Giorno,Mese,Anno,...,Inflazione_cibo,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884.0,500409.900,Monday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0.0,0.000,Tuesday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0.0,0.000,Wednesday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0.0,0.000,Thursday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0.0,0.000,Friday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624.0,19369.152,Tuesday,April,2022,...,6.7,60.5,3614,3915,86,262,62,1469,5650,71843
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0.0,0.000,Wednesday,April,2022,...,6.7,60.5,3592,3804,71,193,27,1160,5945,72033
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723.0,24217.650,Thursday,April,2022,...,6.7,60.5,3838,4031,77,234,29,1528,6001,72263
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0.0,0.000,Friday,April,2022,...,6.7,60.5,3147,3291,73,152,40,976,5197,72488


### Altri passaggi di data preparation e imputazione dati mancanti

In [46]:
# modifica nomi colonne
df_ristorazione = df_ristorazione.rename(columns={'scontrini': 'Scontrini', 'lordototale': 'Lordo_totale'})
df_ristorazione

,Codice,Location,Provincia,Regione,Data,Scontrini,Lordo_totale,Giorno,Mese,Anno,...,Inflazione_cibo,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884.0,500409.900,Monday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0.0,0.000,Tuesday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0.0,0.000,Wednesday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0.0,0.000,Thursday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0.0,0.000,Friday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624.0,19369.152,Tuesday,April,2022,...,6.7,60.5,3614,3915,86,262,62,1469,5650,71843
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0.0,0.000,Wednesday,April,2022,...,6.7,60.5,3592,3804,71,193,27,1160,5945,72033
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723.0,24217.650,Thursday,April,2022,...,6.7,60.5,3838,4031,77,234,29,1528,6001,72263
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0.0,0.000,Friday,April,2022,...,6.7,60.5,3147,3291,73,152,40,976,5197,72488


In [47]:
df_ristorazione.isna().sum()

Codice                     0
Location                   0
Provincia                  0
Regione                    0
Data                       0
Scontrini                  5
Lordo_totale               6
Giorno                     0
Mese                       0
Anno                       0
Stagione                   0
Festività                  0
Festivo_feriale            0
Covid                      0
Colore                     0
Inflazione                 0
Inflazione_cibo            0
Inflazione_combustibili    0
Maschi                     0
Femmine                    0
Prima_dose                 0
Seconda_dose               0
Contagio_precedente        0
Dose_Booster               0
Seconda_dose_Booster       0
Totale_casi                0
dtype: int64

In [48]:
df_ristorazione[df_ristorazione.isna().any(axis=1)]

,Codice,Location,Provincia,Regione,Data,Scontrini,Lordo_totale,Giorno,Mese,Anno,...,Inflazione_cibo,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi
3156,R003,Piacenza,Piacenza,Emilia-Romagna,2022-04-25,472.0,NaN,Monday,April,2022,...,6.7,60.5,70,78,0,3,0,28,117,71759
3400,R005,Voghera,Pavia,Lombardia,2018-08-27,NaN,NaN,Monday,August,2018,...,2.1,5.3,0,0,0,0,0,0,0,0
3401,R005,Voghera,Pavia,Lombardia,2018-08-28,NaN,NaN,Tuesday,August,2018,...,2.1,5.3,0,0,0,0,0,0,0,0
3402,R005,Voghera,Pavia,Lombardia,2018-08-29,NaN,NaN,Wednesday,August,2018,...,2.1,5.3,0,0,0,0,0,0,0,0
3403,R005,Voghera,Pavia,Lombardia,2018-08-30,NaN,NaN,Thursday,August,2018,...,2.1,5.3,0,0,0,0,0,0,0,0
3404,R005,Voghera,Pavia,Lombardia,2018-08-31,NaN,NaN,Friday,August,2018,...,2.1,5.3,0,0,0,0,0,0,0,0


Eliminiamo le righe con dati *NaN*

In [49]:
df_ristorazione = df_ristorazione.dropna(how='any')
df_ristorazione.isna().sum()

Codice                     0
Location                   0
Provincia                  0
Regione                    0
Data                       0
Scontrini                  0
Lordo_totale               0
Giorno                     0
Mese                       0
Anno                       0
Stagione                   0
Festività                  0
Festivo_feriale            0
Covid                      0
Colore                     0
Inflazione                 0
Inflazione_cibo            0
Inflazione_combustibili    0
Maschi                     0
Femmine                    0
Prima_dose                 0
Seconda_dose               0
Contagio_precedente        0
Dose_Booster               0
Seconda_dose_Booster       0
Totale_casi                0
dtype: int64

Ora verifichiamo che tutte le righe con un valore maggiore di 0 per il lordo totale abbiano un numero di scontrini superiore a 0, e viceversa tutte le righe con un numero di scontrini maggiore di 0 in modo che abbiano un lordo totale superiore a 0

In [50]:
df_ristorazione[(df_ristorazione['Lordo_totale'] == 0) & (df_ristorazione['Scontrini'] > 0)]

,Codice,Location,Provincia,Regione,Data,Scontrini,Lordo_totale,Giorno,Mese,Anno,...,Inflazione_cibo,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi
2903,R003,Piacenza,Piacenza,Emilia-Romagna,2021-08-15,602.0,0.0,Sunday,August,2021,...,0.9,23.1,3704,2967,3273,3204,194,0,0,24108
3159,R003,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,491.0,0.0,Thursday,April,2022,...,6.7,60.5,3838,4031,77,234,29,1528,6001,72263
8801,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-15,831.0,0.0,Friday,April,2022,...,6.7,60.5,1725,1440,58,258,8,1781,1060,69894


In [51]:
df_ristorazione[(df_ristorazione['Scontrini'] == 0) & (df_ristorazione['Lordo_totale'] > 0)]

,Codice,Location,Provincia,Regione,Data,Scontrini,Lordo_totale,Giorno,Mese,Anno,...,Inflazione_cibo,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi
821,R000,Montebello,Pavia,Lombardia,2020-04-01,0.0,2.700,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
2402,R003,Piacenza,Piacenza,Emilia-Romagna,2020-04-01,0.0,2.700,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2716
2432,R003,Piacenza,Piacenza,Emilia-Romagna,2020-05-01,0.0,2.700,Friday,May,2020,...,2.9,-11.3,0,0,0,0,0,0,0,4115
3983,R005,Voghera,Pavia,Lombardia,2020-04-01,0.0,2.700,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
5564,R004,Stradella,Pavia,Lombardia,2020-04-01,0.0,2.700,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
5777,R004,Stradella,Pavia,Lombardia,2020-10-31,0.0,23810.166,Saturday,October,2020,...,1.6,-5.7,0,0,0,0,0,0,0,10148
7145,R001,Piacenza,Piacenza,Emilia-Romagna,2020-04-01,0.0,2.700,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2716
7713,R001,Piacenza,Piacenza,Emilia-Romagna,2021-10-21,0.0,20234.718,Thursday,October,2021,...,1.1,26.8,7203,7468,2421,5348,263,6639,0,25641
7714,R001,Piacenza,Piacenza,Emilia-Romagna,2021-10-22,0.0,30487.023,Friday,October,2021,...,1.1,26.8,6763,7044,2046,5299,297,6165,0,25657
7715,R001,Piacenza,Piacenza,Emilia-Romagna,2021-10-23,0.0,36866.097,Saturday,October,2021,...,1.1,26.8,6868,6781,1746,5434,283,6186,0,25671


In questo caso, dobbiamo modificare i valori in modo che corrispondano ad una situazione reale, poichè il valore di scontrino medio che stiamo per calcolare potrebbe essere impreciso a causa di questi errori. 

In [52]:
df_modifica = df_ristorazione.loc[df_ristorazione[(df_ristorazione['Lordo_totale'] == 0) & (df_ristorazione['Scontrini'] > 0)].index]
df_modifica = df_modifica.append(df_ristorazione[(df_ristorazione['Scontrini'] == 0) & (df_ristorazione['Lordo_totale'] > 0)])
df_modifica

C:\Users\andre\AppData\Local\Temp\ipykernel_89568\704973144.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_modifica = df_modifica.append(df_ristorazione[(df_ristorazione['Scontrini'] == 0) & (df_ristorazione['Lordo_totale'] > 0)])


,Codice,Location,Provincia,Regione,Data,Scontrini,Lordo_totale,Giorno,Mese,Anno,...,Inflazione_cibo,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi
2903,R003,Piacenza,Piacenza,Emilia-Romagna,2021-08-15,602.0,0.000,Sunday,August,2021,...,0.9,23.1,3704,2967,3273,3204,194,0,0,24108
3159,R003,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,491.0,0.000,Thursday,April,2022,...,6.7,60.5,3838,4031,77,234,29,1528,6001,72263
8801,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-15,831.0,0.000,Friday,April,2022,...,6.7,60.5,1725,1440,58,258,8,1781,1060,69894
821,R000,Montebello,Pavia,Lombardia,2020-04-01,0.0,2.700,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
2402,R003,Piacenza,Piacenza,Emilia-Romagna,2020-04-01,0.0,2.700,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2716
2432,R003,Piacenza,Piacenza,Emilia-Romagna,2020-05-01,0.0,2.700,Friday,May,2020,...,2.9,-11.3,0,0,0,0,0,0,0,4115
3983,R005,Voghera,Pavia,Lombardia,2020-04-01,0.0,2.700,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
5564,R004,Stradella,Pavia,Lombardia,2020-04-01,0.0,2.700,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
5777,R004,Stradella,Pavia,Lombardia,2020-10-31,0.0,23810.166,Saturday,October,2020,...,1.6,-5.7,0,0,0,0,0,0,0,10148
7145,R001,Piacenza,Piacenza,Emilia-Romagna,2020-04-01,0.0,2.700,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2716


Ora che abbiamo salvato queste righe in un altro dataframe, le rimuoviamo dal nostro dataframe originale

In [53]:
df_ristorazione = df_ristorazione.drop(df_modifica.index)
df_ristorazione

,Codice,Location,Provincia,Regione,Data,Scontrini,Lordo_totale,Giorno,Mese,Anno,...,Inflazione_cibo,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884.0,500409.900,Monday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0.0,0.000,Tuesday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0.0,0.000,Wednesday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0.0,0.000,Thursday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0.0,0.000,Friday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624.0,19369.152,Tuesday,April,2022,...,6.7,60.5,3614,3915,86,262,62,1469,5650,71843
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0.0,0.000,Wednesday,April,2022,...,6.7,60.5,3592,3804,71,193,27,1160,5945,72033
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723.0,24217.650,Thursday,April,2022,...,6.7,60.5,3838,4031,77,234,29,1528,6001,72263
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0.0,0.000,Friday,April,2022,...,6.7,60.5,3147,3291,73,152,40,976,5197,72488


In [54]:
# limitazione a due decimali del lordo totale
df_ristorazione['Lordo_totale'] = df_ristorazione['Lordo_totale'].round(decimals=2)
# variabile scontrini come intero
df_ristorazione['Scontrini'] = df_ristorazione['Scontrini'].astype(int)
df_ristorazione

,Codice,Location,Provincia,Regione,Data,Scontrini,Lordo_totale,Giorno,Mese,Anno,...,Inflazione_cibo,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884,500409.90,Monday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0,0.00,Tuesday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0,0.00,Wednesday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0,0.00,Thursday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0,0.00,Friday,January,2018,...,1.2,5.9,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624,19369.15,Tuesday,April,2022,...,6.7,60.5,3614,3915,86,262,62,1469,5650,71843
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0,0.00,Wednesday,April,2022,...,6.7,60.5,3592,3804,71,193,27,1160,5945,72033
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723,24217.65,Thursday,April,2022,...,6.7,60.5,3838,4031,77,234,29,1528,6001,72263
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0,0.00,Friday,April,2022,...,6.7,60.5,3147,3291,73,152,40,976,5197,72488


In [55]:
df_ristorazione['Scontrino_medio'] = df_ristorazione['Lordo_totale']/df_ristorazione['Scontrini']
df_ristorazione['Scontrino_medio'] = df_ristorazione['Scontrino_medio'].round(decimals=2)
df_ristorazione['Scontrino_medio'] = df_ristorazione['Scontrino_medio'].fillna(0)
df_ristorazione

,Codice,Location,Provincia,Regione,Data,Scontrini,Lordo_totale,Giorno,Mese,Anno,...,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi,Scontrino_medio
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884,500409.90,Monday,January,2018,...,5.9,0,0,0,0,0,0,0,0,22.87
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0,0.00,Tuesday,January,2018,...,5.9,0,0,0,0,0,0,0,0,0.00
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0,0.00,Wednesday,January,2018,...,5.9,0,0,0,0,0,0,0,0,0.00
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0,0.00,Thursday,January,2018,...,5.9,0,0,0,0,0,0,0,0,0.00
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0,0.00,Friday,January,2018,...,5.9,0,0,0,0,0,0,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624,19369.15,Tuesday,April,2022,...,60.5,3614,3915,86,262,62,1469,5650,71843,31.04
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0,0.00,Wednesday,April,2022,...,60.5,3592,3804,71,193,27,1160,5945,72033,0.00
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723,24217.65,Thursday,April,2022,...,60.5,3838,4031,77,234,29,1528,6001,72263,33.50
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0,0.00,Friday,April,2022,...,60.5,3147,3291,73,152,40,976,5197,72488,0.00


Ora che abbiamo calcolato lo scontrino medio, calcoliamo la media di questo valore per l'intero dataframe

In [56]:
df_ristorazione.Scontrino_medio.mean()

22.569769160791452

Ora, per imputare i valori, procediamo in questo modo:
- per i dati con 0 scontrini e lordo totale maggiore di 0, arrotonderemo all'intero più vicino la divisione *Scontrino_medio/Lordo_totale*. 
- per i dati con 0 lordo totale e numero di scontrini maggiore di 0, faremo la moltiplicazione *Scontrini***Scontrino_medio*. 

In [57]:
indici_0scont = [df_modifica[(df_modifica['Lordo_totale'] == 0) & (df_modifica['Scontrini'] > 0)].index]
indici_0scont = [(indici_0scont[0][0], 'Lordo_totale'),(indici_0scont[0][1], 'Lordo_totale'),(indici_0scont[0][2], 'Lordo_totale')]

for index, column in indici_0scont:
    df_modifica.at[index, column] = df_ristorazione.Scontrino_medio.mean()*df_modifica.at[index, 'Scontrini']

df_modifica

,Codice,Location,Provincia,Regione,Data,Scontrini,Lordo_totale,Giorno,Mese,Anno,...,Inflazione_cibo,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi
2903,R003,Piacenza,Piacenza,Emilia-Romagna,2021-08-15,602.0,13587.001035,Sunday,August,2021,...,0.9,23.1,3704,2967,3273,3204,194,0,0,24108
3159,R003,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,491.0,11081.756658,Thursday,April,2022,...,6.7,60.5,3838,4031,77,234,29,1528,6001,72263
8801,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-15,831.0,18755.478173,Friday,April,2022,...,6.7,60.5,1725,1440,58,258,8,1781,1060,69894
821,R000,Montebello,Pavia,Lombardia,2020-04-01,0.0,2.700000,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
2402,R003,Piacenza,Piacenza,Emilia-Romagna,2020-04-01,0.0,2.700000,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2716
2432,R003,Piacenza,Piacenza,Emilia-Romagna,2020-05-01,0.0,2.700000,Friday,May,2020,...,2.9,-11.3,0,0,0,0,0,0,0,4115
3983,R005,Voghera,Pavia,Lombardia,2020-04-01,0.0,2.700000,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
5564,R004,Stradella,Pavia,Lombardia,2020-04-01,0.0,2.700000,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
5777,R004,Stradella,Pavia,Lombardia,2020-10-31,0.0,23810.166000,Saturday,October,2020,...,1.6,-5.7,0,0,0,0,0,0,0,10148
7145,R001,Piacenza,Piacenza,Emilia-Romagna,2020-04-01,0.0,2.700000,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2716


In [58]:
indici_0lord = [df_modifica[(df_modifica['Scontrini'] == 0) & (df_modifica['Lordo_totale'] > 0)].index]
indici_0lord = [(indici_0lord[0][0], 'Scontrini'),(indici_0lord[0][1], 'Scontrini'),(indici_0lord[0][2], 'Scontrini'),
                (indici_0lord[0][3], 'Scontrini'),(indici_0lord[0][4], 'Scontrini'),(indici_0lord[0][5], 'Scontrini'),
                (indici_0lord[0][6], 'Scontrini'),(indici_0lord[0][7], 'Scontrini'),(indici_0lord[0][8], 'Scontrini'),
                (indici_0lord[0][9], 'Scontrini'),(indici_0lord[0][10], 'Scontrini'),(indici_0lord[0][11], 'Scontrini'),
                (indici_0lord[0][12], 'Scontrini'),(indici_0lord[0][13], 'Scontrini')]
for index, column in indici_0lord:
    df_modifica.at[index, column] = df_modifica.at[index, 'Lordo_totale']/df_ristorazione.Scontrino_medio.mean()


df_modifica

,Codice,Location,Provincia,Regione,Data,Scontrini,Lordo_totale,Giorno,Mese,Anno,...,Inflazione_cibo,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi
2903,R003,Piacenza,Piacenza,Emilia-Romagna,2021-08-15,602.000000,13587.001035,Sunday,August,2021,...,0.9,23.1,3704,2967,3273,3204,194,0,0,24108
3159,R003,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,491.000000,11081.756658,Thursday,April,2022,...,6.7,60.5,3838,4031,77,234,29,1528,6001,72263
8801,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-15,831.000000,18755.478173,Friday,April,2022,...,6.7,60.5,1725,1440,58,258,8,1781,1060,69894
821,R000,Montebello,Pavia,Lombardia,2020-04-01,0.119629,2.700000,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
2402,R003,Piacenza,Piacenza,Emilia-Romagna,2020-04-01,0.119629,2.700000,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2716
2432,R003,Piacenza,Piacenza,Emilia-Romagna,2020-05-01,0.119629,2.700000,Friday,May,2020,...,2.9,-11.3,0,0,0,0,0,0,0,4115
3983,R005,Voghera,Pavia,Lombardia,2020-04-01,0.119629,2.700000,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
5564,R004,Stradella,Pavia,Lombardia,2020-04-01,0.119629,2.700000,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
5777,R004,Stradella,Pavia,Lombardia,2020-10-31,1054.958331,23810.166000,Saturday,October,2020,...,1.6,-5.7,0,0,0,0,0,0,0,10148
7145,R001,Piacenza,Piacenza,Emilia-Romagna,2020-04-01,0.119629,2.700000,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2716


Ora arrotondiamo i valori degli scontrini all'intero più vicino (nel caso di valori minori a 1 arrotondiamo a 1), e poniamo i valori di *Lordo_totale* trovati con due cifre decimali come nel resto del dataframe completo. 

In [59]:
df_modifica.loc[df_modifica['Scontrini'] < 1, 'Scontrini'] = 1
df_modifica['Scontrini'] = df_modifica['Scontrini'].astype(int)
df_modifica['Lordo_totale'] = round(df_modifica['Lordo_totale'], 2)
df_modifica

,Codice,Location,Provincia,Regione,Data,Scontrini,Lordo_totale,Giorno,Mese,Anno,...,Inflazione_cibo,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi
2903,R003,Piacenza,Piacenza,Emilia-Romagna,2021-08-15,602,13587.00,Sunday,August,2021,...,0.9,23.1,3704,2967,3273,3204,194,0,0,24108
3159,R003,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,491,11081.76,Thursday,April,2022,...,6.7,60.5,3838,4031,77,234,29,1528,6001,72263
8801,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-15,831,18755.48,Friday,April,2022,...,6.7,60.5,1725,1440,58,258,8,1781,1060,69894
821,R000,Montebello,Pavia,Lombardia,2020-04-01,1,2.70,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
2402,R003,Piacenza,Piacenza,Emilia-Romagna,2020-04-01,1,2.70,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2716
2432,R003,Piacenza,Piacenza,Emilia-Romagna,2020-05-01,1,2.70,Friday,May,2020,...,2.9,-11.3,0,0,0,0,0,0,0,4115
3983,R005,Voghera,Pavia,Lombardia,2020-04-01,1,2.70,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
5564,R004,Stradella,Pavia,Lombardia,2020-04-01,1,2.70,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2180
5777,R004,Stradella,Pavia,Lombardia,2020-10-31,1054,23810.17,Saturday,October,2020,...,1.6,-5.7,0,0,0,0,0,0,0,10148
7145,R001,Piacenza,Piacenza,Emilia-Romagna,2020-04-01,1,2.70,Wednesday,April,2020,...,3.3,-10.4,0,0,0,0,0,0,0,2716


Infine, come per il dataframe originale, calcoliamo la colonna *Scontrino_medio*

In [60]:
df_modifica['Scontrino_medio'] = df_modifica['Lordo_totale']/df_modifica['Scontrini']
df_modifica['Scontrino_medio'] = df_modifica['Scontrino_medio'].round(decimals=2)
df_modifica['Scontrino_medio'] = df_modifica['Scontrino_medio'].fillna(0)
df_modifica

,Codice,Location,Provincia,Regione,Data,Scontrini,Lordo_totale,Giorno,Mese,Anno,...,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi,Scontrino_medio
2903,R003,Piacenza,Piacenza,Emilia-Romagna,2021-08-15,602,13587.00,Sunday,August,2021,...,23.1,3704,2967,3273,3204,194,0,0,24108,22.57
3159,R003,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,491,11081.76,Thursday,April,2022,...,60.5,3838,4031,77,234,29,1528,6001,72263,22.57
8801,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-15,831,18755.48,Friday,April,2022,...,60.5,1725,1440,58,258,8,1781,1060,69894,22.57
821,R000,Montebello,Pavia,Lombardia,2020-04-01,1,2.70,Wednesday,April,2020,...,-10.4,0,0,0,0,0,0,0,2180,2.70
2402,R003,Piacenza,Piacenza,Emilia-Romagna,2020-04-01,1,2.70,Wednesday,April,2020,...,-10.4,0,0,0,0,0,0,0,2716,2.70
2432,R003,Piacenza,Piacenza,Emilia-Romagna,2020-05-01,1,2.70,Friday,May,2020,...,-11.3,0,0,0,0,0,0,0,4115,2.70
3983,R005,Voghera,Pavia,Lombardia,2020-04-01,1,2.70,Wednesday,April,2020,...,-10.4,0,0,0,0,0,0,0,2180,2.70
5564,R004,Stradella,Pavia,Lombardia,2020-04-01,1,2.70,Wednesday,April,2020,...,-10.4,0,0,0,0,0,0,0,2180,2.70
5777,R004,Stradella,Pavia,Lombardia,2020-10-31,1054,23810.17,Saturday,October,2020,...,-5.7,0,0,0,0,0,0,0,10148,22.59
7145,R001,Piacenza,Piacenza,Emilia-Romagna,2020-04-01,1,2.70,Wednesday,April,2020,...,-10.4,0,0,0,0,0,0,0,2716,2.70


A questo punto possiamo reinserire le righe all'interno del nostro dataframe originale

In [61]:
df_ristorazione = df_ristorazione.append(df_modifica).sort_index()
df_ristorazione

C:\Users\andre\AppData\Local\Temp\ipykernel_89568\3227061494.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ristorazione = df_ristorazione.append(df_modifica).sort_index()


,Codice,Location,Provincia,Regione,Data,Scontrini,Lordo_totale,Giorno,Mese,Anno,...,Inflazione_combustibili,Maschi,Femmine,Prima_dose,Seconda_dose,Contagio_precedente,Dose_Booster,Seconda_dose_Booster,Totale_casi,Scontrino_medio
0,R000,Montebello,Pavia,Lombardia,2018-01-01,21884,500409.90,Monday,January,2018,...,5.9,0,0,0,0,0,0,0,0,22.87
1,R000,Montebello,Pavia,Lombardia,2018-01-02,0,0.00,Tuesday,January,2018,...,5.9,0,0,0,0,0,0,0,0,0.00
2,R000,Montebello,Pavia,Lombardia,2018-01-03,0,0.00,Wednesday,January,2018,...,5.9,0,0,0,0,0,0,0,0,0.00
3,R000,Montebello,Pavia,Lombardia,2018-01-04,0,0.00,Thursday,January,2018,...,5.9,0,0,0,0,0,0,0,0,0.00
4,R000,Montebello,Pavia,Lombardia,2018-01-05,0,0.00,Friday,January,2018,...,5.9,0,0,0,0,0,0,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8812,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-26,624,19369.15,Tuesday,April,2022,...,60.5,3614,3915,86,262,62,1469,5650,71843,31.04
8813,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-27,0,0.00,Wednesday,April,2022,...,60.5,3592,3804,71,193,27,1160,5945,72033,0.00
8814,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-28,723,24217.65,Thursday,April,2022,...,60.5,3838,4031,77,234,29,1528,6001,72263,33.50
8815,R002,Piacenza,Piacenza,Emilia-Romagna,2022-04-29,0,0.00,Friday,April,2022,...,60.5,3147,3291,73,152,40,976,5197,72488,0.00


In [62]:
df_ristorazione.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8811 entries, 0 to 8816
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Codice                   8811 non-null   object        
 1   Location                 8811 non-null   object        
 2   Provincia                8811 non-null   object        
 3   Regione                  8811 non-null   object        
 4   Data                     8811 non-null   datetime64[ns]
 5   Scontrini                8811 non-null   int32         
 6   Lordo_totale             8811 non-null   float64       
 7   Giorno                   8811 non-null   object        
 8   Mese                     8811 non-null   object        
 9   Anno                     8811 non-null   object        
 10  Stagione                 8811 non-null   object        
 11  Festività                8811 non-null   object        
 12  Festivo_feriale          8811 non-

### Creazione singoli dataframe per ogni ristorante

In [462]:
ristoranti = ['R000','R001','R002','R003','R004','R005']
rist_list = []
for rist in ristoranti:
    df_rist = df_ristorazione[df_ristorazione['Codice'] == rist]
    rist_list.append(df_rist)

### Esportazione dataframes in formato *csv*

In [463]:
# singoli csv ristoranti
for i, el in enumerate(rist_list):
    file_name = f'dati/rist{i+1}.csv'
    el.to_csv(file_name, index=False)

In [464]:
# csv vaccinazioni
df_vaccinazioni.to_csv('dati/vaccinazioni.csv', index=False)

In [465]:
# csv casi covid
df_casicovid.to_csv('dati/casi_covid.csv', index=False)

In [466]:
# csv colori
df_colori.to_csv('dati/colori.csv', index=False)


In [467]:
# csv inflazione
df_inflazione.to_csv('dati/inflazione.csv', index=False)

In [468]:
# csv dataset completo
df_ristorazione.to_csv('dati/completo.csv', index=False)

### Altro